<a href="https://colab.research.google.com/github/kuznechek/MIPT-homework/blob/dev/HW-1_GameOfBots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Advanced. Функции. Домашнее задание (HW)
## 0. Загрузка и импорт модулей
---

In [2]:
! pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 


In [3]:
import numpy as np
import pandas as pd
import random
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate


# полезные функции
def print_dict(t):
  for key, value in t.items():
    print("{0}: {1}".format(key,value))
  print('\n')

def get_score(first, second):
  if first > second:
    return 1
  elif first < second:
    return -1
  else: return 0

##1. Агенты
---



### Агент № 1 
Всегда выбирает "камень".

In [4]:
def agent_1(observation, configuration):
    return 0

### Агент № 2 
Всегда выбирает "бумагу".

In [5]:
def agent_2(observation, configuration):
    return 1

### Агент № 3
Всегда выбирает "ножницы".

In [6]:
def agent_3(observation, configuration):
    return 2

### Агент № 4
Всегда выбирает случайный ответ.

In [7]:
def agent_4(observation, configuration):
  retValue = random.choice([0, 1, 2])
  return retValue

### Агент № 5
Если на прошлом ходу проиграл, выбирает любое значение, кроме прошлого. Иначе выбирает "бумагу".

In [8]:
lastAct = 1

def agent_5(observation, configuration):
    global lastAct
    if observation.step > 0:
      if observation.reward < 0:
        t = [0, 1, 2]
        t.remove(lastAct)
        lastAct = random.choice(t)
        return lastAct
    else:
      lastAct = 1
    return lastAct

### Агент № 6
Считает, что противник два раза подряд выбрал одинаковое значение, то в третий раз выберет новое.

In [9]:
lastOppAct = [0, 0]
i = 0

def agent_6(observation, configuration):
    global lastOppAct
    global i 
    if observation.step > 0:
      lastOppAct[i] = observation.lastOpponentAction
      i += 1
      i %= 2
      if lastOppAct[0] == lastOppAct[1]:
        if lastOppAct[0] == 0:
          return 2
        elif lastOppAct[0] == 1:
          return 0
    return 1

### Агент № 7
Считает, что противник каждый раз меняет выбор, выбирает ответ, исходя из этой логики.

*(Например, если на прошлом ходу противник выбрал 2 (ножницы), данный агент будет считать, что противник выберет на текущем ходу 0 или 1 (камень или бумагу соответственно). То есть данный агент в такой ситуации выберет  1 (камень).)*

In [10]:
def agent_7(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % 2
    else:
        return random.choice([0, 1, 2])

### Агент № 8
Выбирает между двумя прошлыми значениями оппонента. Если прошло менее двух кругов, выбирает "ножницы".

In [11]:
lastOppAct = [0, 0]
i = 0

def agent_8(observation, configuration):
    global lastOppAct
    global i 
    if observation.step > 0:
      lastOppAct[i] = observation.lastOpponentAction
      i += 1
      i %= 2
      return random.choice(lastOppAct)
    return 2

### Агент № 9
Повторяет действия оппонента, если выиграл. В других ситуациях выбирает случайное значение.

In [26]:
lastAct = 0

def agent_9(observation, configuration):
    global lastAct
    if observation.step == 0:
      lastAct = random.choice([0, 1, 2])
    elif get_score(lastAct, observation.lastOpponentAction) == -1:
      lastAct = observation.lastOpponentAction
    else: 
      lastAct = random.choice([0, 1, 2])
    return lastAct

### Агент № 10
Выкидывает значение, которое победило в прошлом раунде. В случае ничьи выкидывает случайное значение.

In [13]:
lastAct = 0

def agent_10(observation, configuration):
    global lastAct
    if observation.step == 0:
      lastAct = random.choice([0, 1, 2])
    elif get_score(lastAct, observation.lastOpponentAction) == 1:
      lastAct = observation.lastOpponentAction
    elif get_score(observation.lastOpponentAction, lastAct) == 0:
      lastAct = random.choice([0, 1, 2])
    return lastAct

### Агент № 11
Выбирает по очереди: "камень", "бумагу", нрожницы".

In [14]:
c = -1

def agent_11(observation, configuration):
    global c
    c += 1
    if c % 3 == 0:
        return 0
    elif c % 3 == 1:
        return 1
    else:
        return 2

### Агент № 12
Выбирает случайное число и выкидывает его, пока не проиграет. Затем выбирает другое случайное.

In [15]:
lastAct = random.choice([0, 1, 2])

def agent_12(observation, configuration):
    global lastAct
    if observation.step > 0:
      if get_score(lastAct, observation.lastOpponentAction) == -1:
        lastAct = random.choice([0, 1, 2])
    return lastAct

### Агент № 13
Выбирает случайное значение, затем повторяет действия оппонента, затем снова выбирает случайное значение, и так далее по кругу.

In [16]:
i = 0

def agent_13(observation, configuration):
    global i
    if i % 2 == 0:
      return random.choice([0, 1, 2])
    else:
      return observation.lastOpponentAction

## 2. Запуск игр и вывод результата
---
Запуск игр производится в соответствии со списком агентов *agents*.

Каждый агент описан приведнной выше функцией и возвращает значение 0, 1 или 2 в зависимости от вложенной логики агента.

In [35]:
agents = [
    agent_1,
    agent_2,
    agent_3,
    agent_4,
    agent_5,
    agent_6,
    agent_7,
    agent_8,
    agent_9,
    agent_10,
    agent_11,
    agent_12,
    agent_13
]

results = {}
count = 0

def test_all_agents(n_rounds=1):
  scores = {agent.__name__: 0 for agent in agents}
  l = len(agents)
  for i in range(l):
    for j in range(i+1, l):
      player_1, player_2 = agents[i], agents[j]

      result = evaluate( "rps", [player_1, player_2],
                        configuration={'episodeSteps': 100,
                                       'tieRewardThreshold': 1},
                        num_episodes=n_rounds )
      for k in range(n_rounds):
        if result[0][0] > result[0][1]:
          scores[player_1.__name__] += 1
        else:
          scores[player_2.__name__] += 1
  return scores

results = test_all_agents(5)
print_dict(results)

agent_1: 20
agent_2: 25
agent_3: 25
agent_4: 20
agent_5: 30
agent_6: 10
agent_7: 30
agent_8: 30
agent_9: 40
agent_10: 25
agent_11: 50
agent_12: 50
agent_13: 35




In [37]:
finTab = pd.Series(results, name="Рейтинг агентов по количеству побед").sort_values(ascending=False).to_frame()
finTab

,Рейтинг агентов по количеству побед
agent_11,50
agent_12,50
agent_9,40
agent_13,35
agent_5,30
agent_7,30
agent_8,30
agent_2,25
agent_3,25
agent_10,25
